In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import sympy as sp
sp.init_printing(use_unicode=True)

In [219]:
kappa = 1
delta_x = 0.2
delta_t = 0.02
lam = kappa*delta_t/delta_x**2
L = lam**2

t_init = 0
t_target = 0.04
x_init = -0.2
x_target = 1

l = 1

def u(x, t):
    # The order in which these boundary conditions are defined is important!
    if t == 0:
        return x*(1-x)  
    if x == l:
        return 0
    if x == 0:
        return 0

def du(x, t):
    if t == 0:
        return 0
    if x == 0:
        return 1
    
forcing_term = 0
    
dims = (int(np.round((t_target-t_init)/delta_t))+1, int(np.round((x_target-x_init)/delta_x+1)))

In [236]:
a = np.array([-lam, 2*(1+lam), -lam])
A = np.eye(dims[1])
for i in range(1, dims[1]-1):
    A[i][i-1:i+2] = a
A[0][0] = -1
A[0][2] = 1
A_inv = np.linalg.inv(A)
sp.Matrix(A)

⎡-1.0  0.0   1.0   0.0   0.0   0.0   0.0 ⎤
⎢                                        ⎥
⎢-0.5  3.0   -0.5  0.0   0.0   0.0   0.0 ⎥
⎢                                        ⎥
⎢0.0   -0.5  3.0   -0.5  0.0   0.0   0.0 ⎥
⎢                                        ⎥
⎢0.0   0.0   -0.5  3.0   -0.5  0.0   0.0 ⎥
⎢                                        ⎥
⎢0.0   0.0   0.0   -0.5  3.0   -0.5  0.0 ⎥
⎢                                        ⎥
⎢0.0   0.0   0.0   0.0   -0.5  3.0   -0.5⎥
⎢                                        ⎥
⎣0.0   0.0   0.0   0.0   0.0   0.0   1.0 ⎦

In [237]:
b = np.array([lam, 2*(1-lam), lam])
C = np.eye(dims[1])
for i in range(1, dims[1]-1):
    C[i][i-1:i+2] = b
C[0][0] = 0
sp.Matrix(C)

⎡0.0  0.0  0.0  0.0  0.0  0.0  0.0⎤
⎢                                 ⎥
⎢0.5  1.0  0.5  0.0  0.0  0.0  0.0⎥
⎢                                 ⎥
⎢0.0  0.5  1.0  0.5  0.0  0.0  0.0⎥
⎢                                 ⎥
⎢0.0  0.0  0.5  1.0  0.5  0.0  0.0⎥
⎢                                 ⎥
⎢0.0  0.0  0.0  0.5  1.0  0.5  0.0⎥
⎢                                 ⎥
⎢0.0  0.0  0.0  0.0  0.5  1.0  0.5⎥
⎢                                 ⎥
⎣0.0  0.0  0.0  0.0  0.0  0.0  1.0⎦

In [238]:
grid = np.zeros(dims)

In [239]:
grid[0] = [u(x, 0) for x in delta_x*np.arange(x_init/delta_x, dims[1]-1)]
grid[0][1:dims[1]-1] = grid[0][1:dims[1]-1] + delta_x**2*forcing_term/2*np.ones((1, dims[1]-2))

# grid[:,0] = [u(0, t) for t in np.arange(dims[0])]
# grid[:,dims[1]-1] = [u(l, t) for t in delta_t*np.arange(dims[0])]
sp.Matrix(np.round(grid, decimals=4))

⎡-0.24  0.0  0.16  0.24  0.24  0.16  0.0⎤
⎢                                       ⎥
⎢ 0.0   0.0  0.0   0.0   0.0   0.0   0.0⎥
⎢                                       ⎥
⎣ 0.0   0.0  0.0   0.0   0.0   0.0   0.0⎦

In [243]:
rhs = np.zeros((1, dims[1]))
rhs[0][0] = u(0.2, 0) - u(-0.2, 0)
for i in range(1, dims[0]):
    grid[i] = A_inv.dot((C.dot(grid[i-1])+rhs).T).T

In [244]:
sp.Matrix(np.round(grid, decimals=5))

⎡ -0.24      0.0      0.16     0.24     0.24     0.16    0.0⎤
⎢                                                           ⎥
⎢-0.27996  -0.03999  0.12004  0.2002   0.20118  0.12686  0.0⎥
⎢                                                           ⎥
⎣-0.31972  -0.07989  0.08028  0.16126  0.16569  0.10343  0.0⎦